# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 75
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecastPolling
import postOptimization
import numpy as np

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()
    
day_in_minute_interval_steps = list(np.around(np.arange(480, 960 + 1, minute_interval), 1))
distributionAlgorithmForecastPolling.init_simulation(day_in_minute_interval_steps, minute_interval, simulation_data, 
                                                   simulation_day, solarpeakleistung)

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                ladeleistung_pro_bev)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)

START POST OPTIMIZATION
charging list per minute dict:  {}


OPTIMIZATION DONE
BEVs dict:  {0: [(8.0, 4.5), 'wartend', [(691.500684957718, 15.056823808358672, 0)], [3.6599999999999993]], 1: [(9.5, 6.4), 'nicht parkend', [], [5.189999999999999]], 2: [(9.0, 3.3), 'nicht parkend', [], [3.3475]], 3: [(8.5, 4.2), 'nicht parkend', [], [3.7399999999999998]], 4: [(8.5, 5.3), 'nicht parkend', [], [4.6825]], 5: [(9.0, 6.6), 'nicht parkend', [], [5.4174999999999995]], 6: [(12.5, 3.5), 'wartend', [(947.2974308300395, 12.702569169960498, 0)], [2.1425]], 7: [(12.5, 3.5), 'wartend', [(917.540505723385, 29.756925106654535, 0)], [2.1425]], 8: [(8.0, 6.6), 'nicht parkend', [], [5.329999999999998]], 9: [(8.5, 3.6), 'nicht parkend', [], [3.285]], 10: [(12.5, 3.5), 'wartend', [(925.1162790697674, 19.883720930232585, 0)], [2.1425]], 11: [(8.0, 3.3), 'nicht parkend', [], [2.5649999999999995]], 12: [(8.5, 6.6), 'nicht parkend', [], [5.437499999999999]], 13: [(13.0, 3.0), 'wartend', [(904.6885185826948, 12.851

SET BEV DATA for current minute for ID BEV 8 for minute 525:  8.19
SET BEV DATA for current minute for ID BEV 11 for minute 525:  8.19
Charging BEVs (nachdem neue Plätze belegt):  [8, 11]
Update currently charging bevs
Number of charging bevs:  2
AVAILABLE SOLAR POWER:  16.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 8:  2.317499999999998
Restladezeit:  16.978021978021964
Update currently charging bevs
Number of charging bevs:  2
AVAILABLE SOLAR POWER:  16.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 11:  1.1749999999999996
Restladezeit:  8.608058608058606
Die Restladezeit des BEVS 11 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 11 for minute 533.6080586080586:  8.38
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  18
Ladestart für ausgewähltes BEV:  533.6080586080586
SET BEV DATA for charging start of new bev for ID BEV 18 for minute 533.6080586080586:  8.19
SET BEV DATA charging start between intervals for I

SET BEV DATA for current minute for ID BEV 40 for minute 585:  9.4
SET BEV DATA for current minute for ID BEV 60 for minute 585:  9.4
SET BEV DATA for current minute for ID BEV 61 for minute 585:  9.4
SET BEV DATA for current minute for ID BEV 65 for minute 585:  9.4
Charging BEVs (nachdem neue Plätze belegt):  [28, 40, 60, 61, 65]
Update currently charging bevs
Number of charging bevs:  5
AVAILABLE SOLAR POWER:  47.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 28:  1.4883333333333315
Restladezeit:  9.499999999999988
Die Restladezeit des BEVS 28 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 28 for minute 594.5:  9.614
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  68
Ladestart für ausgewähltes BEV:  594.5
SET BEV DATA for charging start of new bev for ID BEV 68 for minute 594.5:  9.4
SET BEV DATA charging start between intervals for ID BEV 68 for minute 594.5:  9.4
Update currently charging bevs
Number of charging bevs:  5
AVA

SET BEV DATA for current minute for ID BEV 12 for minute 630:  10.076
SET BEV DATA for current minute for ID BEV 14 for minute 630:  10.076
SET BEV DATA for current minute for ID BEV 24 for minute 630:  10.076
SET BEV DATA for current minute for ID BEV 26 for minute 630:  10.076
Charging BEVs (nachdem neue Plätze belegt):  [71, 12, 14, 24, 26]
Update currently charging bevs
Number of charging bevs:  5
AVAILABLE SOLAR POWER:  50.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 71:  0.21583333333331467
Restladezeit:  1.285232235013783
Die Restladezeit des BEVS 71 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 71 for minute 631.2852322350138:  10.096
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  42
Ladestart für ausgewähltes BEV:  631.2852322350138
SET BEV DATA for charging start of new bev for ID BEV 42 for minute 631.2852322350138:  10.076
SET BEV DATA charging start between intervals for ID BEV 42 for minute 631.2852322350138:  

Charging BEVs after updating post optimization plan and parking end:  [5, 27, 36, 41]
charging_list_per_minute_dict  {480: [8], 495: [8], 510: [8, 11], 525: [8, 11], 540: [8, 18, 20], 555: [18, 20, 22, 72], 570: [20, 28, 40, 60], 585: [28, 40, 60, 61, 65], 600: [65, 68, 71, 3, 4], 615: [68, 71, 3, 4, 9], 630: [71, 12, 14, 24, 26], 645: [12, 24, 26, 42, 66], 660: [66, 69, 73, 74, 2]}
LAAASSST minute:  660
NUUUUUUMBER of charging bevs of last interval:  5
SET BEV DATA for current minute for ID BEV 5 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 27 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 36 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 41 for minute 675:  10.376000000000001
Charging BEVs (nachdem neue Plätze belegt):  [5, 27, 36, 41]
Update currently charging bevs
Number of charging bevs:  5
AVAILABLE SOLAR POWER:  51.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 5:  5.1

Waiting BEVs:  [1, 48, 49, 43]
SET BEV DATA after charging time over for ID BEV 52 for minute 735:  10.05
Removed from charging list because of overload:  52
Charging list after handling overload:  [53, 63, 64, 62]
SET BEV DATA charging start between intervals for ID BEV 50 for minute 741.1918401445793:  9.576
BEV 52 CHARGING END FROM POST OPTI in minute
Waiting BEVs after updating parking end:  [1, 48, 49, 43]
Charging BEVs after updating post optimization plan and parking end:  [53, 63, 64, 62]
charging_list_per_minute_dict  {480: [8], 495: [8], 510: [8, 11], 525: [8, 11], 540: [8, 18, 20], 555: [18, 20, 22, 72], 570: [20, 28, 40, 60], 585: [28, 40, 60, 61, 65], 600: [65, 68, 71, 3, 4], 615: [68, 71, 3, 4, 9], 630: [71, 12, 14, 24, 26], 645: [12, 24, 26, 42, 66], 660: [66, 69, 73, 74, 2], 675: [5, 27, 36, 41, 38], 690: [41, 38, 0, 54, 57], 705: [57, 55, 52, 53, 63], 720: [52, 53, 63, 64, 62]}
LAAASSST minute:  720
NUUUUUUMBER of charging bevs of last interval:  5
FUEL Charging BEV:  

SET BEV DATA after charging time over for ID BEV 34 for minute 795:  9.375
Removed from charging list because of overload:  34
SET BEV DATA after charging time over for ID BEV 37 for minute 795:  9.375
Removed from charging list because of overload:  37
Charging list after handling overload:  [44, 45]
SET BEV DATA charging start between intervals for ID BEV 30 for minute 799.9807486631016:  9.5625
BEV 34 CHARGING END FROM POST OPTI in minute
BEV 37 CHARGING END FROM POST OPTI in minute
SET BEV DATA charging start between intervals for ID BEV 39 for minute 805.6570890049812:  9.5625
BEV 44 CHARGING END FROM POST OPTI in next interval
SET BEV DATA after charging time over for ID BEV 44 for minute 799.9807486631016:  9.5
BEV 45 CHARGING END FROM POST OPTI in next interval
SET BEV DATA after charging time over for ID BEV 45 for minute 805.6570890049812:  9.6425
Waiting BEVs after updating parking end:  [70]
Charging BEVs after updating post optimization plan and parking end:  [44, 45]
char

SET BEV DATA after charging time over for ID BEV 16 for minute 855:  12.0
Removed from charging list because of overload:  16
Charging list after handling overload:  [31]
BEV 16 CHARGING END FROM POST OPTI in minute
SET BEV DATA charging start between intervals for ID BEV 23 for minute 867.2863636363636:  11.0
SET BEV DATA charging start between intervals for ID BEV 25 for minute 860.9863636363636:  11.0
BEV 31 CHARGING END FROM POST OPTI in minute
Waiting BEVs after updating parking end:  [67]
Charging BEVs after updating post optimization plan and parking end:  [31]
charging_list_per_minute_dict  {480: [8], 495: [8], 510: [8, 11], 525: [8, 11], 540: [8, 18, 20], 555: [18, 20, 22, 72], 570: [20, 28, 40, 60], 585: [28, 40, 60, 61, 65], 600: [65, 68, 71, 3, 4], 615: [68, 71, 3, 4, 9], 630: [71, 12, 14, 24, 26], 645: [12, 24, 26, 42, 66], 660: [66, 69, 73, 74, 2], 675: [5, 27, 36, 41, 38], 690: [41, 38, 0, 54, 57], 705: [57, 55, 52, 53, 63], 720: [52, 53, 63, 64, 62], 735: [53, 63, 64, 6

BEV 10 CHARGING END FROM POST OPTI in minute
Waiting BEVs after updating parking end:  []
Charging BEVs after updating post optimization plan and parking end:  []
charging_list_per_minute_dict  {480: [8], 495: [8], 510: [8, 11], 525: [8, 11], 540: [8, 18, 20], 555: [18, 20, 22, 72], 570: [20, 28, 40, 60], 585: [28, 40, 60, 61, 65], 600: [65, 68, 71, 3, 4], 615: [68, 71, 3, 4, 9], 630: [71, 12, 14, 24, 26], 645: [12, 24, 26, 42, 66], 660: [66, 69, 73, 74, 2], 675: [5, 27, 36, 41, 38], 690: [41, 38, 0, 54, 57], 705: [57, 55, 52, 53, 63], 720: [52, 53, 63, 64, 62], 735: [53, 63, 64, 62, 50], 750: [53, 50, 1, 51, 58], 765: [58, 48, 46, 47], 780: [34, 37, 44, 45], 795: [44, 45, 30, 39], 810: [30, 39, 70, 35], 825: [56, 29, 32, 33], 840: [33, 59, 16, 31], 855: [31, 23, 25], 870: [25, 19, 21], 885: [21, 17], 900: [13, 15], 915: [7, 10], 930: [7, 10]}
LAAASSST minute:  930
NUUUUUUMBER of charging bevs of last interval:  2
Charging BEVs (nachdem neue Plätze belegt):  []
Charging BEVs:  []
BEVs 

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag, solarpeakleistung, minute_interval)

>**Ergebnis**<br>Geladene BEVs: 74 von 75<br>Verfügbare Solarenergie insgesamt: 276.47 kWh<br>Aufgeladene Solarenergie insgesamt: 195.75 kWh<br>Ungenutzte Solarenergie insgesamt: 80.72 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_bev_number_figure(simulation_data)

In [7]:
figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)